In [1]:
import pandas as pd
import numpy as np

import pickle
data_test =pd.read_pickle('/Users/emmawebster/Desktop/SDS/data_288.pkl')
print(len(data_test))
data_test.head()

288


,Address,Car description,Car model,Colour,Energy label,KM driven,KM per litre,Manufacture year,Price new,Price used,Tank size litre,Top speed km/h,Type of gear
0,Følfodvej 3 9310 Vodskov,7 PERSONERS.Automatgear 12V strømudtag 2-zoner...,"Nissan Qashqai 2 2,0 dCi Tekna aut 4WD 5d",Hvid,energimaerke energimaerke-d,169.000,14.1,2011,482.489,174.700,65,186,Automatisk
1,Trianglen 11 6000 Kolding,URBAN pakke navigation klimaanlæg lys syns pa...,"Mercedes A180 1,5 CDi Urban 5d",Brunmetal,energimaerke energimaerke-a0,25.000,25.0,2015,-,269.900,50,190,Manuel
2,Industrivej 2 9280 Storvorde,Luksuriøs Audi A6 med MEGET udstyr Havannaschw...,"Audi A6 2,8 FSi 204 Avant Multitr 5d",Sortmetal,energimaerke energimaerke-d,122.000,13.5,2014,724.283,499.900,65,230,Automatisk
3,Grenåvej 347 8240 Risskov,DK-Pakke fuldaut. klima fjernb. c.lås startsp...,"VW Up 1,0 MPi 60 Move Up BMT 5d",Blå,energimaerke energimaerke-a0,7.000,24.4,2017,111.998,109.700,35,162,Manuel
4,Høegh Guldbergs Gade 48 8700 Horsens,"263 KML LED-forlygter 16"" alufælge fuldaut. KL...","Seat Leon 1,6 TDi 105 Style ST 5d",Hvid,energimaerke energimaerke-a0,107.000,26.3,2014,272.402,169.500,50,191,Manuel


In [2]:
data_test['description']=data_test['Car description']
text=data_test['description']

In [3]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)

print_full(data_test['description'])

0      7 PERSONERS.Automatgear 12V strømudtag 2-zoner...
1      URBAN pakke navigation klimaanlæg lys  syns pa...
2      Luksuriøs Audi A6 med MEGET udstyr Havannaschw...
3       DK-Pakke fuldaut. klima fjernb. c.lås startsp...
4      263 KML LED-forlygter 16" alufælge fuldaut. KL...
5      KAN KØBES UDEN UDBETALING airc. navigation mul...
6       aut. 18" alufælge 2 zone klima fjernb. c.lås ...
7      KAN KØBES UDEN UDBETALING airc. fjernb. c.lås ...
8      Pure white - hvid komplet sort læder adaptive ...
9      Multimediesystem MediaNav m. navigation og 7" ...
10     aut.geartiptronic 18" alufælge 2 zone klima kø...
11     Fantastisk rummelig st.car. med utolig gode kø...
12      nissan connect navigation med bakkamera 7" sk...
13      airc. køl i handskerum fjernb. c.lås parkerin...
14     Pure Vision Full LED forlygter inkl tågelygter...
15     6 gear 2 zone klima køl i handskerum fjernb. c...
16     Det viste tilbud er inkl. 2 års garanti og 2 å...
17      16" alufælge airc. fjer

In [88]:



dictionary = {'Klimavenlig': set(['langt på literen', 'elbil', 'miljø', 'Tesla', 'miljøvenlig', 'elmotor', 'miljø', 'natur',''])
              ,'Luksus': set(['læderbetræk', 'soltag', 'luksus', 'eftertragtet', 'luksusiøs', 'sports-model', 'BMW', 'Audi', 'sportsvogn'])}

# Define function that checks if the words are in that sentence.
import nltk # our tokenizer and stemmer
nltk.download('punkt') # download tokenizers!! 

stemmer_dk = nltk.stem.snowball.DanishStemmer()
stemmer_gb = nltk.stem.snowball.EnglishStemmer()

def preprocess(text):
    
    ### Cleaning the text. Skipping that for now.
    # Example could be removing html. Only allow certain set of characters.
    
    ### To lowercase
    text = text.lower()
    # Tokenize
    wordlist = nltk.word_tokenize(text)
    # stem
    wordlist = [stemmer_dk.stem(w) and stemmer_gb.stem(w) for w in wordlist]
    return wordlist

def classification_by_dictionary(text, dictionary=dictionary):
    ## make_sure it is a text
    if type(text)!=str: 
        return np.nan
    
     # Cleaning and tokenization
    text = set(preprocess(text)) # Converting to set
    topics = []
    for topic in dictionary:
        topiclist = dictionary[topic] # set of words indicative of the topic
        overlap = topiclist&text
        if len(overlap)>0:
            topics.append(topic)
    return tuple(topics)


data_test['topics'] = text.apply(classification_by_dictionary) # Running this on >100.000 posts might take some time (minutes).
print(data_test.topics.value_counts())
 

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/emmawebster/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
()                276
(Luksus,)          11
(Klimavenlig,)      1
Name: topics, dtype: int64


In [72]:
import sklearn
import re
def custom_tokenize(text):
    # Clean text: Typos, bad grammar, usage of slang, presence of unwanted content like URLs,
        # remove urls
    text = re.sub(r'http.*','',text)
    
    # tokenize and stem
    wordlist = preprocess(text) # our old function
    wordlist = [word.strip('.,"?') for word in wordlist]
    return wordlist
vectorizer = sklearn.feature_extraction.text.CountVectorizer(tokenizer=custom_tokenize)
texts = text[0:100].dropna()
bows = vectorizer.fit_transform(texts) # fit and transform as opposed to just transform.
## Ready as input.
bows

<100x1360 sparse matrix of type '<class 'numpy.int64'>'
	with 6602 stored elements in Compressed Sparse Row format>

In [73]:
### BoW representation as input to Gensim
## Gensims own implementation
# First you create a dictionary, mapping word to index
import gensim
documents = [custom_tokenize(doc) for doc in text[0:100].dropna()]
dictionary = gensim.corpora.Dictionary(documents)
corpus = [dictionary.doc2bow(doc) for doc in documents] # Corpus is a list of BoWs
corpus[50]

# Corpus is a list of BoWs #dictionary.token2id

[(11, 1),
 (13, 1),
 (15, 1),
 (20, 1),
 (21, 1),
 (29, 1),
 (35, 1),
 (46, 1),
 (47, 1),
 (48, 1),
 (49, 1),
 (51, 1),
 (71, 5),
 (77, 1),
 (85, 1),
 (86, 1),
 (87, 1),
 (88, 1),
 (89, 1),
 (90, 1),
 (91, 1),
 (92, 1),
 (93, 1),
 (94, 1),
 (95, 1),
 (97, 1),
 (98, 1),
 (99, 1),
 (100, 1),
 (115, 1),
 (123, 1),
 (165, 1),
 (333, 1),
 (380, 1),
 (493, 1),
 (1029, 1),
 (1030, 1),
 (1031, 1),
 (1032, 1),
 (1033, 1)]

In [74]:
### Sklearn version
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(tokenizer=custom_tokenize)
texts = text[0:100].dropna()
tfidf = vectorizer.fit_transform(texts) # fit and transform as opposed to just transform.
tfidf
### Gensims own implementation
tfidf_transformer = gensim.models.TfidfModel(corpus,id2word = dictionary.id2token)
tfidfs = tfidf_transformer[corpus]